# Introduction

In the last notebook, we started to explore the data, mainly dealing with airports and flights. Now, let's move on to exploring when flights operate. One thing that you will find in working with data is that there are a lot of ways that one can use the data you are working with. As an analyst, it is up to you to develop an understanding of the end goal of your analysis that can help you get more specific about your analysis questions and then figure out the best way to use the data sources available to answer those questions. This is often more difficult than the technical parts of the analysis.

For instance, the question "When do flights operate?" can be broken down in many different ways. How do flight operations vary by time of year? What about time of day? Do domestic vs. international flights operate at different times? And so on and so forth.

We will tackle a few different angles on this broad question. So without further ado, let's start exploring!

# Import & Ingest

In [ ]:
# Import Libraries

import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import plotly.express as px
import calendar

Let's start by importing our dataset from last time called eda_data.csv.

In [ ]:
# Answer
filepath = r'/home/jovyan/eda_data.csv'
df = pd.read_csv(filepath)

# Exploratory Data Analysis

#### How has COVID impacted air travel?

Let's start with system-level disruption. The past year has been an unprecedented one. The COVID pandemic has been felt around the world. Every person and every industry has had to adjust to this new reality. The travel industry has been no exception. For an initial analysis, let's explore the effect COVID has had on the airline industry in the United States. Hopefully, at the end of this notebook, we will be able to better summarize and understand how the airlines and airports have been affected.  

As you have noticed, the time column is a combination of date and time. Python labels this type of data as datatime. Though time is interesting, let's first just work with the dates. We can do this by creating a new column using the date() method (accessed by .dt). 

In [ ]:
df['day_of_flight'] = pd.to_datetime(df['time']).dt.date

We will first plot the total number of flights by day. Using what we learned in the previous notebook, why don't you give this a try?

In [ ]:
# Answer

df_flights = df.groupby(['day_of_flight'])['airport'].count().reset_index(name = 'count_of_flights')

plt.plot(df_flights['day_of_flight'], df_flights['count_of_flights'])
plt.title("Daily Flight Counts")
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.ylabel('# of Flights')
plt.gca().set_ylim(ymin=0)
plt.show()

Wow, look at this graph? What are some initial observations

We immediately see a huge drop in the middle of March. This is when the the seriousness of the COVID pandemic first became clear in the US. In the thick of the pandemic during the early summer months of April and May, we see that the volume was roughly 4000 flights per day, which is 25% - 30% of pre-Covid volumes. We also see that there was a slow rise until the first week of July. From there, we see a steady volume of about 8000 flights per day - a little over half of pre-covid volumes. 

One thing to notice is that the day-to-day fluctactions add noise to the chart. Depending on your analysis goals, those fluctuations could be of specific interest (what is behind those occasional big drops?) or could be a distraction. For our purposes, let's try rolling up the daily counts into weekly counts to see if that cleans up the graph a little bit and makes it easier to see broader trends.

We first need to define a new column called weeks and then group the observations by week. 

In [ ]:
#Answer 

df['week'] = pd.to_datetime(df['day_of_flight']) - pd.offsets.Week(weekday=1)
df_flights_week = df.groupby('week')['airport'].count().reset_index(name = 'count_of_flights') 

Now let's plot those weekly counts.

In [ ]:
plt.plot(df_flights_week['week'], df_flights_week['count_of_flights'])
plt.title("Weekly Flight Counts")
plt.xlabel('Week Start')
plt.gca().set_ylim(ymin=0)
plt.ylabel('# of Flights')
plt.xticks(rotation=45)
plt.show()

This graph is much cleaner, and it is easier to see the trends of the number of flights over time. One odd thing to notice is that there is a dip in the observation week observation. This is weird because we didn't see this in the day level view. This is a commom problem when grouping dates. In this case, this is because the dataset ends in the middle of a week, so the last count does not include a full seven days of flights. You can confirm this on your own.

Because this group is incomplete, let us just remove it and replot the graph. 

In [ ]:
# Answer

df_flights_week_clean = df_flights_week[df_flights_week['week'] < '2020-08-25']
plt.plot(df_flights_week_clean['week'], df_flights_week_clean['count_of_flights'])
plt.title("Weekly Flight Counts")
plt.xlabel('Week Start')
plt.gca().set_ylim(ymin=0)
plt.ylabel('# of Flights')
plt.xticks(rotation=45)
plt.show()

Now that we have looked at the effects of COVID at a high level, take a closer look at how different subsets of flights have been impacted.

#### Has COVID affected domestic and international flights differently?

To answer this question, let's start by trying to recreate similar time series line plots as we did above - first for domestic flights, then for international flights. This time, it'll all up to you!

In [ ]:
# Answer
df_flight_time_clean = df[df['week'] < '2020-08-25']
df_local = df_flight_time_clean[df_flight_time_clean['type_of_flight'] == 'local'].groupby(
    'week')['airport'].count().reset_index(name = 'count_of_flights')

plt.plot(df_local['week'], df_local['count_of_flights'],  label = 'local')
plt.title("Weekly Flight Counts - Domestic")
plt.xlabel('Week Start')
plt.gca().set_ylim(ymin=0)
plt.ylabel('# of Flights')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Answer
df_international = df_flight_time_clean[df_flight_time_clean['type_of_flight'] == 'international'].groupby('week')['airport'].count().reset_index(name = 'count_of_flights')
plt.plot(df_international['week'], df_international['count_of_flights'],  label = 'international')
plt.title("Weekly Flight Counts - International")
plt.xlabel('Week Start')
plt.gca().set_ylim(ymin=0)
plt.ylabel('# of Flights')
plt.xticks(rotation=45)
plt.show()

Both domestic and international flights follow a similar trend at the highest level, but it looks like the recovery may have been slower for international flights.

To make a more direct comparison, let's plot them on the same graph. 

In [ ]:
plt.plot(df_local['week'], df_local['count_of_flights'],  label = 'domestic')
plt.plot(df_international['week'], df_international['count_of_flights'],  label = 'international')
plt.title("Weekly Flight Counts")
plt.xlabel('Week Start')
plt.gca().set_ylim(ymin=0)
plt.ylabel('# of Flights')
plt.xticks(rotation=45)
plt.legend()
plt.show()

Before commenting on the graph, one thing that you will notice from the code above is that to plot two sets of data on the same graph with matplotlib, you can effectively write two plot statements, and matplotlib will superimpose them onto each other. 

Now let's think about this graph. This combined graph is actually not as useful as the individual graphs - we can see that there are many fewer international flights throughout the data set, but it is difficult to compare trends due to the difference in scale. In this case, much of the detail for international flights is obscured by the much higher volume of domestic flights. So what might be a better way to plot these two sets of data? 

One possible solution would be using a normalized metric such as the percent change in flight volumes relative to some pre-COVID baseline time period. This would allow us to compare the relative drop in flights and relative recoveries between the two groups. To do this, we would need to select an appropriate baseline time period, compute the average weekly flights for each group during the baseline period, and then divide the number of weekly flights over the rest of the time period by that baseline average. 

Let's make the baseline everything prior to March 1. This exercise is a little bit harder but why don't you give it a try using all that you have learned up to this point. Note: you can compute averages using the mean() method. 

How would you interpret this graph?

In [ ]:
# Answer
from matplotlib.ticker import PercentFormatter

df_local['percent_change'] = df_local['count_of_flights'] / df_local[df_local['week'] < '2020-3-1']['count_of_flights'].mean() - 1
df_international['percent_change'] = df_international['count_of_flights'] / df_international[df_international['week'] < '2020-3-1']['count_of_flights'].mean() - 1

plt.plot(df_local['week'], df_local['percent_change'], label = 'domestic')
plt.plot(df_international['week'], df_international['percent_change'], label = 'international')
plt.title("Percent Change of Weekly Flight Counts from Start of Year")
plt.xlabel('Week Start')
plt.gca().set_ylim(ymin=-1)
plt.ylabel('Percent Change')
plt.xticks(rotation=45)
plt.legend()
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.show()

Looking at this graph, we can see when the initial wave of flight cancellations hit, international flight volume was down more than 90% from the first 2 months of the year compared to a drop of less than 80% for domestic flights. Moreover, while domestic flights have recovered about half of their decline, international flights are still down almost 70%. Does this match your intuition? What are some possible reasons for this discrepancy, and how could you evaluate those reasons if you had other data available?

Now let's see if COVID has affected airlines equally. 

#### How has COVID impacted airlines differently?

This question is similar to the previous question. However, we now have 4 airlines to consider compared to only 2 types of flights (domestic and international) in the earlier analysis. It would be quite cumbersome to have to copy and paste all the code and change a few columns here and there to complete the analysis for each airline. In fact, if we were dealing with a dataset with more attributes, rewriting the same code over and over would quickly become unmanageable.

This is where one of the advantages of code-based analysis comes into play. Instead of rewriting code, we can create something called a function. Effectively, functions are blocks of code that can be used to consolidate repeated functionality. The function can be called with a single line of code each time that functionality is needed. A function can take arguments (or input parameters) that can modify what data is considered or tweak the specific functionality.

In this section, we have provided a function definition for you. You will need to run the below code block to define the function and make it available to be called later in the notebook

In [ ]:
def analyze_and_plot(df, drilldown_column):
    df['week'] = pd.to_datetime(df['day_of_flight']) - pd.offsets.Week(weekday=1)
    df_flight = df.groupby(['week', drilldown_column])['airport'].count().reset_index(name = 'count_of_flights')
    df_clean = df_flight[df_flight['week'] < '2020-08-25']
    
    fig = px.line(df_clean, x="week", y="count_of_flights", color=drilldown_column,
                  labels = dict(week = "Week", count_of_flights = "Count of Flights"),
                 title = "Weekly Flight Counts by " + drilldown_column)
    fig.show()

To explain the code above, you'll notice that it starts off with the def keyword which indicates that we are going define a new function. *analyze_and_plot* is the name of the new function, and *df* and *drilldown_column* are the input parameters. The remaining code is what the function will do each time it is called. Most of the code should be familiar as we used it previously. Note where the function is using the input parameters.

To use this function, all we need to do is call it. You can see this being done below.

In [ ]:
analyze_and_plot(df, 'airline')

Let's also define a function for the percent change in flight volumes.

In [ ]:
def analyze_and_plot_percent(df, drilldown_column):
    df['week'] = pd.to_datetime(df['day_of_flight']) - pd.offsets.Week(weekday=1)
    df_flight = df.groupby(['week', drilldown_column])['airport'].count().reset_index(name = 'count_of_flights')
    df_clean = df_flight[df_flight['week'] < '2020-08-25']
    first_value = df_clean[df_clean['week'] < '2020-3-1'][[drilldown_column,'count_of_flights']].groupby(drilldown_column).mean().rename(columns = {'count_of_flights': 'first_value'})

    df_merged = df_clean.merge(first_value, on = [drilldown_column])
    df_merged['percent_change'] = df_merged['count_of_flights'] / df_merged['first_value'] - 1
    
    fig = px.line(df_merged, x="week", y="percent_change", color=drilldown_column, 
                  labels = dict(week = "Week", percent_change = "Percent Change"),
                 title = "Percent Change of Weekly Flight Counts from Start of Year by " + drilldown_column)
    fig.update_layout(yaxis = dict(tickformat="%"))
    fig.show()
    
    return df_merged

One addition to this new function that we added was a return statement. This specifies what information will be passed back from the function when it is called. In this case, it is the final dataframe that we used for plotting. We can now use this dataframe for other analysis.

In [ ]:
df_airline = analyze_and_plot_percent(df, 'airline')

So here we see that United Airlines was hit the hardest and has not recovered as quickly as the other airlines while Southwest saw the smalled relative drop in the initial shutdown and has recovered to within 20% of its flight volumes from the beginning of the year.

#### Which airport has COVID affected the most? The least?

The great thing about functions is that they can be reused without having to copy and paste the code over and over again. Let's use the functions from the last section to see if we can answer this question.

In [ ]:
# Anaylze 

df_airport = analyze_and_plot_percent(df, 'airport')

You'll immediately notice that the number of lines on the graph makes it extremely hard to read. In fact, the only information you are able to glean are really any airports that are outliers. 

These graphs were plotted using a library called plotly. One great thing about these graphs is that if you click the various airports in the legend, you can toggle whether or not you want each airline to be shown on the graph. So that should help mitigate the problem. However, it is still hard to identify which airports have been the most affected or least affected. Let's compute some summary metrics on how each airport has been impacted.

In [ ]:
df_airport['week'] = pd.to_datetime(df_airport['week'])
df_airport['time_period'] = 'Recovery'
df_airport.loc[df_airport['week'] < dt.datetime(2020,7,1), 'time_period'] = 'Early_Covid'
df_airport.loc[df_airport['week'] < dt.datetime(2020,3,15), 'time_period'] = 'Before_Covid'

df_airport_avg = df_airport.groupby(['airport','time_period'])['percent_change'].mean().reset_index(name = 'Relative_Traffic')
df_airport_avg = df_airport_avg.pivot(index = 'airport', columns = 'time_period')['Relative_Traffic'].reset_index()[['airport','Before_Covid','Early_Covid','Recovery']]

# Airports that have recovered the best
df_airport_avg.sort_values('Recovery', ascending = False).head(5)

From this data, we can also answer questions like which airport saw the biggest initial impact from COVID on flight volumes. In fact, let's answer that. 

In [ ]:
# Answer
df_airport_avg.sort_values('Early_Covid').head(5)

The major New York City airports (JFK, Newark, and LaGuardia) are at the top of the list with the biggest initial drop in flight volumes, which is consistent with how heavily that region was hit early in the pandemic.

So far, we have focused on trends over time and the macro effect of COVID on air traffic volumes. There are other ways to look at flights over time as well. Let's look at some other perspectives. 

#### How do flight volumes vary by day of week?

Since COVID drastically altered flight frequencies, let's simplify this exercise by just looking at the Pre-Covid months. 

In [ ]:
df_precovid = df[df['week'] < '2020-03-15'].copy()
df_precovid['weekday'] = pd.to_datetime(df_precovid['time']).dt.weekday
df_precovid['week_day'] = df_precovid.weekday.apply(lambda x: str(x) + ' ' + calendar.day_name[x][:3])
df_precovid = df_precovid.sort_values(['weekday'])

To compare values between discrete categories (in this case, between days of the week), use a bar chart. First, let's look at overall flight counts by day.

In [ ]:
daily_counts = df_precovid.groupby(['week_day'])['week_day'].count().reset_index(name = 'count')
plt.bar(daily_counts['week_day'], daily_counts['count'] / df_precovid['week'].nunique())
plt.title('Average Daily Flight Count - Pre-COVID')
plt.show()

Not a huge swing over the course of a week, but clearly more flights operating at either end of the work week with a dip over the weekend. I wonder if certain flights are driving this pattern. What if we just look at the domestic flights?

In [ ]:
# Domestic
daily_counts_local = df_precovid[df_precovid['type_of_flight'] == 'local'].groupby(
    ['week_day'])['week_day'].count().reset_index(name = 'count')

plt.bar(daily_counts_local['week_day'], daily_counts_local['count'] / df_precovid['week'].nunique())
plt.title('Average Domestic Flight Count - Pre-COVID')
plt.show()

Very similar pattern. Try creating a similar bar chart for international flights.

In [ ]:
# Answer

# International
daily_counts_intl = df_precovid[df_precovid['type_of_flight'] == 'international'].groupby(
    ['week_day'])['week_day'].count().reset_index(name = 'count')

plt.bar(daily_counts_intl['week_day'], daily_counts_intl['count'] / df_precovid['week'].nunique())
plt.title('Average International Flight Count - Pre-COVID')
plt.show()

International flights seem to increase on the weekend - the opposite of what we saw for domestic flights! For a better comparison, let's try to plot them on the same graph. We'll use some pandas trickery to create a grouped bar chart...

In [ ]:
pd.concat([daily_counts_local['count'].rename('domestic') / df_precovid['week'].nunique(), 
           daily_counts_intl['count'].rename('international') / df_precovid['week'].nunique()], 
          axis=1).set_index(daily_counts_local['week_day']).plot.bar().legend(loc=2);
plt.title('Average Daily Flight Count - Pre-COVID')
plt.show()

Hmm...we seem to have run into the same problem as with the line plots. Because there are so many more domestic flights, it is hard to make the comparison that we were hoping to between relative flight volumes. In this case, a percent change doesn't make much sense, but one possible comparison is the relative proportion of flights by day within each group rather than raw counts. 

In [ ]:
pd.concat([daily_counts_local['count'].rename('domestic') / daily_counts_local['count'].sum(),
           daily_counts_intl['count'].rename('international') / daily_counts_intl['count'].sum()], 
          axis=1).set_index(daily_counts_local['week_day']).plot.bar().legend(loc=3);
plt.title('Distribution of Flights by Weekday - Pre-COVID')
plt.show()

Now, looking at the graph, the contrast is much clearer - international flights skew toward weekends while domestic flight volumes are higher during the week.

#### When do flights fly out during the day?

Now that we have looked at how to look at flight by times during the week. How would we do this for the time of flights during the day? And what conclusion can we draw from the data?

In [ ]:
# Answer

df_precovid['hour'] = pd.to_datetime(df_precovid['time']).dt.hour

hourly_counts_local = df_precovid[df_precovid['type_of_flight'] == 'local'].groupby(
    ['hour'])['hour'].count().reset_index(name = 'count')
hourly_counts_intl = df_precovid[df_precovid['type_of_flight'] == 'international'].groupby(
    ['hour'])['hour'].count().reset_index(name = 'count')

pd.concat([hourly_counts_local['count'].rename('domestic')/hourly_counts_local['count'].sum(),
           hourly_counts_intl['count'].rename('international')/hourly_counts_intl['count'].sum()], 
          axis=1).set_index(hourly_counts_local['hour']).plot.bar().legend(loc=2);
plt.title('Distribution of Flights by Hour (GMT) - Pre-COVID')
plt.show()

Looking at this graph, it is really interesting the difference between domestic and international flights. Other than the lull of the overnight period (remember - these are GMT hours), the flight volumes are fairly constant. But international flights, there are two primary volume peaks: one in the morning and one in the night. Is this expected?

# Conclusion

We've explored a few different angles for analysis, but so far we've limited ourselves to using just the flight event data. Next up: we'll explore how to integrate other data to enhance the analysis.